In [1]:
import json
from datetime import datetime, date, timedelta
import pandas as pd
import pyspark.sql.functions as F
from cafu.metadata import path
path_odds = path('datalake')+'/odds/franca/2021-2022/angers_vs_brest.json'
r = open(path_odds, 'r')
odds = json.load(r)

In [2]:
odds

{'Vitória / Empate / Vitória, Tempo Regular': {'Angers': '2.44',
  'Empate': '3.15',
  'Brest': '3.30'},
 'Vitória / Empate / Vitória, Primeiro Tempo': {'Angers': '3.10',
  'Empate': '2.07',
  'Brest': '3.95'},
 'Vitória / Empate / Vitória, Segundo Tempo': {'Angers': '2.70',
  'Empate': '2.40',
  'Brest': '3.40'},
 'Handicap Asiático, Tempo Regular': {'Angers -1,-1.5': '4.22',
  'Brest +1,+1.5': '1.23',
  'Angers -1': '3.70',
  'Brest +1': '1.29',
  'Angers -0.5,-1': '2.78',
  'Brest +0.5,+1': '1.48',
  'Angers -0.5': '2.33',
  'Brest +0.5': '1.67',
  'Angers 0,-0.5': '2.01',
  'Brest 0,+0.5': '1.91',
  'Angers 0': '1.63',
  'Brest 0': '2.40',
  'Angers 0,+0.5': '1.49',
  'Brest 0,-0.5': '2.75'},
 'Handicap Asiático, Primeiro Tempo': {'Angers -0.5,-1': '4.12',
  'Brest +0.5,+1': '1.22',
  'Angers 0,-0.5': '2.38',
  'Brest 0,+0.5': '1.62',
  'Angers 0': '1.70',
  'Brest 0': '2.25',
  'Angers 0,+0.5': '1.40',
  'Brest 0,-0.5': '3.00'},
 'Mais de / Menos de - Brest, Tempo Regular': {'Mais

In [7]:
odds['date_update']

'2022-03-20 09:21:07.984072'

In [10]:
with open('teste/odds.json', 'w') as fp:
    json.dump(odds, fp)

In [12]:
time_casa = 'Olympique Marselha'
time_visitante = 'Nice'
(
time_casa+
' vs ' +
time_visitante+
'.json'
).replace(' ','_').lower()

'olympique_marselha_vs_nice.json'

In [4]:
from cafu.utils import get_spark
spark = get_spark(1)

22/03/19 19:21:18 WARN Utils: Your hostname, luan-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/03/19 19:21:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/luan/.ivy2/cache
The jars for the packages stored in: /home/luan/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/spark-3.0.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9d65f58f-fccf-42e0-8010-06940185d32c;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in cen

In [3]:
from cafu.utils import proximas_partidas
from cafu.metadata import campeonato_dafabet
from cafu.queries import GetOdds
from datetime import datetime, date, timedelta
from unidecode import unidecode

In [4]:
# campeonatos que terão jogos, hoje ou amanhã
pr_part = proximas_partidas()
dafabet = campeonato_dafabet()
after_tomorrow = date.today()+timedelta(days=2)
campeonatos = []
for c in pr_part:
    for t in pr_part[c]:
        try:
            prox_jogo = datetime.strptime(pr_part[c][t], '%Y-%m-%d').date()
            if (prox_jogo<=after_tomorrow) and (c in dafabet):
                campeonatos.append([c,t])
        except:
            pass

In [5]:
pr_part

{'brasil': {'2021-2021': 'finalizado',
  '2020-2020': 'finalizado',
  '2019-2019': 'finalizado',
  '2018-2018': 'finalizado',
  '2017-2017': 'finalizado',
  '2016-2016': 'finalizado',
  '2015-2015': 'finalizado',
  '2014-2014': 'finalizado'},
 'espanha': {'2021-2022': '2022-03-19',
  '2020-2021': 'finalizado',
  '2019-2020': 'finalizado',
  '2018-2019': 'finalizado',
  '2017-2018': 'finalizado',
  '2016-2017': 'finalizado',
  '2015-2016': 'finalizado',
  '2014-2015': 'finalizado',
  '2013-2014': 'finalizado',
  '2012-2013': 'finalizado',
  '2011-2012': 'finalizado'},
 'franca': {'2021-2022': '2022-03-19',
  '2020-2021': 'finalizado',
  '2019-2020': 'finalizado'}}

In [5]:
query = GetOdds()
for c in campeonatos:
    stop = False
    index = 0
    while not stop:
        query.get_campeonato_dafabet(c[0])
        descricao_partida = query.join_link_odds_partida(index)
        if descricao_partida is not None:
            if (
                ('Hoje' in descricao_partida['horario']) or
                ('Amanhã' in descricao_partida['horario'])
               ):
                query.open_odds()
                odds = query.get_odds()
                for k1 in odds:
                    for k2 in odds[k1]:
                        odds[k1][k2] = float(odds[k1][k2])
                for k1 in odds:
                    df = spark.createDataFrame([odds[k1]])
                    df = (
                             df
                             .withColumn('time_casa', F.lit(descricao_partida['time_casa']))
                             .withColumn('time_visitante', F.lit(descricao_partida['time_visitante']))
                             .withColumn('horario', F.lit(descricao_partida['horario']))
                             .withColumn('campeonato_metadata', F.lit(c))
                             .withColumn('temporada_metadata', F.lit(t))
                             .withColumn('date_update', F.lit(datetime.now()))
                         )
                    for c in df.columns:
                        df = df.withColumnRenamed(c, (c.replace(' ','_')
                                                       .replace(',','_'))
                                                 )
                    dir_ = k1.replace('/','-')
                    dir_ = unidecode(dir_.lower())
                    df.write.parquet(f'teste/{dir_}', mode='append')
            else:
                stop = True
        else:
            stop = True
        index+=1
query.web.close() # encerra a sessão

 10%|█         | 1/10 [00:05<00:49,  5.52s/it]


In [9]:
z = odds, 4

In [11]:
len(z[0])

77

In [1]:
from cafu.etl import update_odds

In [ ]:
update_odds(spark)

100%|██████████| 14/14 [00:05<00:00,  2.57it/s]
0it [00:00, ?it/s]
  2%|▏         | 9/409 [00:26<18:36,  2.79s/it]